# Nemotron-3 Nano 30B Fine-tuning on Clarity Dataset

This notebook fine-tunes the Nemotron-3 Nano 30B model on the Frenzyknight/clarity-dataset for political discourse clarity classification.

**Task**: 3-class classification (Clear Reply, Ambivalent, Clear Non-Reply)

Run this on an A100 GPU (e.g., Google Colab Pro or similar).

### Installation

In [ ]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch==2.7.1" "triton>=3.3.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth"
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

# These are mamba kernels and we must have these for faster training
# Mamba kernels are for now supported only on torch==2.7.1. If you have newer torch versions, please wait 30 minutes for it to compile
!uv pip install --no-build-isolation mamba_ssm==2.2.5
!uv pip install --no-build-isolation causal_conv1d==1.5.2

### Load Nemotron Model

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Nemotron-3-Nano-30B-A3B",
    max_seq_length = 2048,  # Sufficient for clarity classification task
    load_in_4bit = False,   # 4 bit quantization to reduce memory
    load_in_8bit = False,   # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False,# [NEW!] We have full finetuning now!
    trust_remote_code = True,
    unsloth_force_compile = True,
    attn_implementation="eager",
    # token = "hf_...", # use one if using gated models
)

### Add LoRA Adapters

We add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # Rank - higher for more capacity. Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "in_proj", "out_proj",],
    lora_alpha = 32,
    lora_dropout = 0,  # Supports any, but = 0 is optimized
    bias = "none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth",  # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

### Load Clarity Dataset

We load the Frenzyknight/clarity-dataset which contains political interview Q&A pairs labeled for clarity classification.

The dataset has a `conversations` column with the format:
```
[
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "Clear Reply" | "Ambivalent" | "Clear Non-Reply"}
]
```

In [ ]:
from datasets import load_dataset

# Load the clarity dataset from HuggingFace
dataset = load_dataset("Frenzyknight/clarity-dataset", split="train")

print(f"Dataset size: {len(dataset)} examples")
print(f"Columns: {dataset.column_names}")
print(f"\nSample conversation:")
print(dataset[0]['conversations'])

### Apply Chat Template

We apply the Nemotron chat template to the conversations and save to the `text` field.

Nemotron uses the following format:
```
<|im_start|>system
...<|im_end|>
<|im_start|>user
...<|im_end|>
<|im_start|>assistant
...<|im_end|>
```

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, 
            tokenize=False, 
            add_generation_prompt=False
        ) 
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
# Let's see how the chat template formatted our data
print("Sample formatted text:")
print("="*80)
print(dataset[0]['text'][:2000])  # Print first 2000 chars
print("...")

### Configure Training

We use the SFTTrainer from TRL with optimized settings for the clarity classification task.

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,  # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,  # Effective batch size = 4 * 2 = 8
        warmup_steps = 10,
        num_train_epochs = 3,  # Train for 3 epochs on clarity dataset
        # max_steps = 100,  # Uncomment for quick test run
        learning_rate = 2e-4,  # Standard for LoRA fine-tuning
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use "wandb" for Weights & Biases logging
        save_strategy = "epoch",
    ),
)

### Train on Responses Only

We use Unsloth's `train_on_responses_only` to only train on the assistant outputs (the classification labels) and ignore the loss on the user's inputs. This helps increase accuracy!

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
# Verify masking is working - the input should be masked (shown as spaces)
print("Full input:")
print(tokenizer.decode(trainer.train_dataset[0]["input_ids"])[:1000])
print("\n" + "="*80)
print("\nMasked labels (only assistant response should be visible):")
labels = trainer.train_dataset[0]["labels"]
decoded = tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in labels])
print(decoded.replace(tokenizer.pad_token, " ")[-500:])  # Show end where label is

### Check GPU Memory

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

### Train the Model

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# Show final memory stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_training = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
training_percentage = round(used_memory_for_training / max_memory * 100, 3)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_training} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage}%.")
print(f"Peak reserved memory for training % of max memory = {training_percentage}%.")

### Inference - Test the Model

Let's test our fine-tuned model on a sample from the clarity task.

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Sample test prompt
test_messages = [
    {
        "role": "system",
        "content": "You are an expert political discourse analyst specializing in classifying response clarity in political interviews. Your task is to determine whether a politician's response to a specific question is a Clear Reply, Clear Non-Reply, or Ambivalent."
    },
    {
        "role": "user",
        "content": """Based on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.

### Classification Categories ###
1. Clear Reply - The information requested is explicitly stated (in the requested form)
2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification, or declining to answer
3. Ambivalent - The information requested is given in an incomplete way (e.g., the answer is too general, partial, implicit, dodging, or deflection)

### Full Interview Question ###
Do you support the new healthcare bill?

### Full Interview Answer ###
Well, healthcare is certainly important to all Americans. We need to make sure that everyone has access to quality care. There are many aspects of this bill that we're still reviewing.

### Specific Question to Classify ###
Do you support the new healthcare bill?

Classify the response clarity for this specific question. Respond with only one of: Clear Reply, Clear Non-Reply, or Ambivalent"""
    }
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 0.1,
    do_sample = True,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Model Response:")
print("="*80)
# Extract just the assistant's response
if "assistant" in response:
    print(response.split("assistant")[-1].strip())
else:
    print(response[-200:])

### Save the Model

Save the LoRA adapters locally and optionally push to HuggingFace Hub.

In [ ]:
# Save locally
model.save_pretrained("nemotron_clarity_lora")
tokenizer.save_pretrained("nemotron_clarity_lora")
print("Model saved to nemotron_clarity_lora/")

In [ ]:
# Optional: Push to HuggingFace Hub
# Uncomment and fill in your details to push

# model.push_to_hub(
#     "your-username/nemotron-clarity-lora",
#     token = "hf_...",  # Your HuggingFace token
# )
# tokenizer.push_to_hub(
#     "your-username/nemotron-clarity-lora",
#     token = "hf_...",
# )

### Save as Merged Model (Optional)

If you want to save the full merged model (base + LoRA) for easier deployment:

In [ ]:
# Save merged 16-bit model (larger but easier to use)
# model.save_pretrained_merged("nemotron_clarity_merged", tokenizer, save_method="merged_16bit")

# Or save as GGUF for llama.cpp
# model.save_pretrained_gguf("nemotron_clarity_gguf", tokenizer, quantization_method="q4_k_m")

---
## Full Test Set Inference & Evaluation

Run inference on the complete test dataset, compute metrics, and save results to CSV.

In [ ]:
# Load test set
test_dataset = load_dataset("Frenzyknight/clarity-dataset", split="test")
print(f"Test set size: {len(test_dataset)} examples")
print(f"Columns: {test_dataset.column_names}")

In [ ]:
import pandas as pd
from tqdm import tqdm
import re

# Label normalization function
def normalize_label(text):
    """Normalize model output to standard labels."""
    if not text:
        return "Unknown"
    
    text_lower = text.lower().strip()
    
    # Check for exact matches first
    if "clear reply" in text_lower and "non" not in text_lower:
        return "Clear Reply"
    elif "clear non-reply" in text_lower or "clear non reply" in text_lower:
        return "Clear Non-Reply"
    elif "ambivalent" in text_lower:
        return "Ambivalent"
    
    # Fallback patterns
    if text_lower.startswith("clear reply"):
        return "Clear Reply"
    elif text_lower.startswith("clear non"):
        return "Clear Non-Reply"
    elif text_lower.startswith("ambivalent"):
        return "Ambivalent"
    
    return "Unknown"

def extract_prediction(response_text):
    """Extract the prediction from model response."""
    # Try to find assistant response
    if "<|im_start|>assistant" in response_text:
        pred = response_text.split("<|im_start|>assistant")[-1]
        pred = pred.replace("<|im_end|>", "").strip()
    elif "assistant" in response_text.lower():
        pred = response_text.split("assistant")[-1].strip()
    else:
        # Take the last part of the response
        pred = response_text.strip()
    
    # Clean up
    pred = pred.strip().split("\n")[0]  # Take first line only
    return pred

print("Label normalization test:")
print(f"  'Clear Reply' -> {normalize_label('Clear Reply')}")
print(f"  'clear non-reply' -> {normalize_label('clear non-reply')}")
print(f"  'Ambivalent.' -> {normalize_label('Ambivalent.')}")

In [ ]:
# Run inference on full test set
FastLanguageModel.for_inference(model)

results = []

for idx, example in enumerate(tqdm(test_dataset, desc="Running inference")):
    convo = example['conversations']
    
    # Extract ground truth label (assistant response)
    ground_truth = None
    for msg in convo:
        if msg['role'] == 'assistant':
            ground_truth = msg['content']
            break
    
    # Prepare inference conversation (without assistant response)
    inference_convo = [msg for msg in convo if msg['role'] != 'assistant']
    
    # Tokenize and generate
    inputs = tokenizer.apply_chat_template(
        inference_convo,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=32,
            use_cache=True,
            temperature=0.1,
            do_sample=False,  # Greedy decoding for consistency
            pad_token_id=tokenizer.pad_token_id,
        )
    
    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    raw_prediction = extract_prediction(full_response)
    normalized_prediction = normalize_label(raw_prediction)
    normalized_ground_truth = normalize_label(ground_truth) if ground_truth else "Unknown"
    
    # Extract user prompt for context
    user_content = ""
    for msg in convo:
        if msg['role'] == 'user':
            user_content = msg['content'][:500]  # Truncate for CSV
            break
    
    results.append({
        'idx': idx,
        'ground_truth': ground_truth,
        'ground_truth_normalized': normalized_ground_truth,
        'raw_prediction': raw_prediction,
        'prediction': normalized_prediction,
        'correct': normalized_prediction == normalized_ground_truth,
        'user_prompt': user_content,
    })
    
    # Progress update every 50 examples
    if (idx + 1) % 50 == 0:
        current_acc = sum(r['correct'] for r in results) / len(results)
        print(f"  Progress: {idx+1}/{len(test_dataset)} | Current Accuracy: {current_acc:.2%}")

print(f"\nInference complete! Processed {len(results)} examples.")

In [ ]:
# Convert to DataFrame
results_df = pd.DataFrame(results)

# Display sample results
print("Sample Results:")
print("="*80)
display(results_df[['idx', 'ground_truth_normalized', 'prediction', 'correct', 'raw_prediction']].head(10))

### Compute Evaluation Metrics

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Filter out unknown predictions for metrics
valid_results = results_df[
    (results_df['prediction'] != 'Unknown') & 
    (results_df['ground_truth_normalized'] != 'Unknown')
].copy()

print(f"Total examples: {len(results_df)}")
print(f"Valid examples (excluding Unknown): {len(valid_results)}")
print(f"Unknown predictions: {len(results_df) - len(valid_results)}")

# Calculate accuracy
if len(valid_results) > 0:
    y_true = valid_results['ground_truth_normalized'].tolist()
    y_pred = valid_results['prediction'].tolist()
    
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\n{'='*50}")
    print(f"ACCURACY: {accuracy:.2%} ({sum(valid_results['correct'])}/{len(valid_results)})")
    print(f"{'='*50}")

In [ ]:
# Classification Report
if len(valid_results) > 0:
    labels = ['Clear Reply', 'Ambivalent', 'Clear Non-Reply']
    
    print("\nClassification Report:")
    print("="*60)
    print(classification_report(y_true, y_pred, labels=labels, zero_division=0))

In [ ]:
# Confusion Matrix
if len(valid_results) > 0:
    labels = ['Clear Reply', 'Ambivalent', 'Clear Non-Reply']
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    print("\nConfusion Matrix:")
    print("="*60)
    print(f"{'':20} {'Predicted':^45}")
    print(f"{'':20} {'Clear Reply':^15} {'Ambivalent':^15} {'Clear Non-Reply':^15}")
    print("-"*65)
    for i, label in enumerate(labels):
        print(f"{label:20} {cm[i][0]:^15} {cm[i][1]:^15} {cm[i][2]:^15}")
    
    # Also show as heatmap if matplotlib available
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels, yticklabels=labels)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix - Nemotron Clarity Classification')
        plt.tight_layout()
        plt.show()
    except ImportError:
        print("\n(Install matplotlib and seaborn for confusion matrix visualization)")

In [ ]:
# Per-class accuracy breakdown
if len(valid_results) > 0:
    print("\nPer-Class Accuracy:")
    print("="*50)
    for label in labels:
        class_df = valid_results[valid_results['ground_truth_normalized'] == label]
        if len(class_df) > 0:
            class_acc = class_df['correct'].sum() / len(class_df)
            print(f"  {label:20}: {class_acc:.2%} ({class_df['correct'].sum()}/{len(class_df)})")
        else:
            print(f"  {label:20}: N/A (no examples)")

In [ ]:
# Show prediction distribution
print("\nPrediction Distribution:")
print("="*50)
print(results_df['prediction'].value_counts())

print("\nGround Truth Distribution:")
print("="*50)
print(results_df['ground_truth_normalized'].value_counts())

### Save Results to CSV

In [ ]:
# Save full results
output_csv = "nemotron_clarity_predictions.csv"
results_df.to_csv(output_csv, index=False)
print(f"Results saved to: {output_csv}")

# Also save a summary
summary = {
    'total_examples': len(results_df),
    'valid_examples': len(valid_results),
    'accuracy': accuracy if len(valid_results) > 0 else 0,
    'unknown_predictions': (results_df['prediction'] == 'Unknown').sum(),
}

print("\n" + "="*50)
print("EVALUATION SUMMARY")
print("="*50)
for k, v in summary.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.4f}")
    else:
        print(f"  {k}: {v}")

In [ ]:
# Show some incorrect predictions for analysis
incorrect = results_df[~results_df['correct']].head(10)
if len(incorrect) > 0:
    print("\nSample Incorrect Predictions (for analysis):")
    print("="*80)
    for _, row in incorrect.iterrows():
        print(f"\nExample {row['idx']}:")
        print(f"  Ground Truth: {row['ground_truth_normalized']}")
        print(f"  Predicted:    {row['prediction']}")
        print(f"  Raw Output:   {row['raw_prediction'][:100]}...")
        print("-"*40)

### Download Results (Colab)

In [ ]:
# Download CSV (for Google Colab)
try:
    from google.colab import files
    files.download(output_csv)
    print(f"Downloaded: {output_csv}")
except:
    print(f"CSV saved locally: {output_csv}")
    print("(Run in Colab to auto-download)")